<a href="https://colab.research.google.com/github/zealair12/CV-for-OCR---PwC/blob/main/Label_Studio_Annotations_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install libraries

In [ ]:
!apt-get update
!apt-get install -y tesseract-ocr


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:4 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [ ]:
!pip install pytesseract
!pip install pillow

In [ ]:
!tesseract --version


tesseract 4.1.1
 leptonica-1.82.0
  libgif 5.1.9 : libjpeg 8d (libjpeg-turbo 2.1.1) : libpng 1.6.37 : libtiff 4.3.0 : zlib 1.2.11 : libwebp 1.2.2 : libopenjp2 2.4.0
 Found AVX2
 Found AVX
 Found FMA
 Found SSE
 Found libarchive 3.6.0 zlib/1.2.11 liblzma/5.2.5 bz2lib/1.0.8 liblz4/1.9.3 libzstd/1.4.8


## Code files

In [ ]:
import pytesseract
from PIL import Image
import json
import requests
from io import BytesIO

In [ ]:
# Path to the image
# for loan worksheet
image_url = 'https://allies-assets.s3.us-east-1.amazonaws.com/birthplan_builder_assets/extra_images/LenderFeesWorksheetNew_pages.jpg'

# for invoice
#image_url = 'https://allies-assets.s3.us-east-1.amazonaws.com/birthplan_builder_assets/extra_images/invoice_sample.png'

In [ ]:
# Path to tesseract executable
pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'

# Fetch the image from the URL
response = requests.get(image_url)
if response.status_code == 200:
    image = Image.open(BytesIO(response.content))
else:
    print(f"Error fetching image. HTTP status code: {response.status_code}")
    exit()

# Get image dimensions
image_width, image_height = image.size

# Get image dimensions
image_width, image_height = image.size

# Perform OCR with Tesseract
ocr_data = pytesseract.image_to_data(image, output_type=pytesseract.Output.DICT)

In [ ]:
# Generate bounding boxes and text for OCR results
ocr_results = []
for i in range(len(ocr_data['text'])):
    if ocr_data['text'][i].strip():  # Skip empty text entries
        # Convert bbox dimensions to percentages
        x_min = (ocr_data['left'][i] / image_width) * 100
        y_min = (ocr_data['top'][i] / image_height) * 100
        width = (ocr_data['width'][i] / image_width) * 100
        height = (ocr_data['height'][i] / image_height) * 100

        ocr_results.append({
            'text': ocr_data['text'][i],
            'bbox': [x_min, y_min, width, height],
            'confidence': ocr_data['conf'][i]
        })


In [ ]:
# Save OCR results to JSON
ocr_json_path = "ocr_results.json"
with open(ocr_json_path, 'w') as f:
    json.dump(ocr_results, f, indent=4)

print(f"OCR results saved to {ocr_json_path}")

OCR results saved to ocr_results.json


In [ ]:
# Prepare Label Studio JSON format
label_studio_annotations = []
for idx, entry in enumerate(ocr_results):
    bbox = entry["bbox"]
    label_studio_annotations.append({
        "id": str(idx),  # Unique ID for each annotation
        "type": "rectanglelabels",
        "from_name": "label",
        "to_name": "image",
        "image_rotation": 0,
        "original_width": image_width,
        "original_height": image_height,
        "value": {
            "x": bbox[0],  # Top-left x in percentages
            "y": bbox[1],  # Top-left y in percentages
            "width": bbox[2],  # Width in percentages
            "height": bbox[3],  # Height in percentages
            "rotation": 0,  # Assuming no rotation
            "rectanglelabels": ["Detected Text"]  # Single label for the bounding box
        }
    })

In [ ]:
# Generate the final Label Studio format
final_output = [
    {
        "data": {
            "image": image_url # Path to image
        },
        "predictions": [
            {
                "model_version": "v1",
                "score": 0.9,  # Example score for the prediction
                "result": label_studio_annotations
            }
        ]
    }
]

# Save the Label Studio JSON
output_path = "label_studio_output.json"
with open(output_path, "w") as f:
    json.dump(final_output, f, indent=4)

print(f"Label Studio annotation JSON saved to {output_path}")


Label Studio annotation JSON saved to label_studio_output.json
